In [4]:
from landslide4sense.models import Unet
from landslide4sense.data import LandslideDataSet
from landslide4sense.data import LandslideDataSet
from landslide4sense.utils.tools import eval_image
from landslide4sense.models import Unet
from torch.utils.data import DataLoader
import torch
import omegaconf
import matplotlib.pyplot as plt
from torch.nn import Upsample, CrossEntropyLoss

In [3]:
cfg = omegaconf.OmegaConf.load("../conf/config.yaml")
cfg

{'data': {'dir': '${hydra:runtime.cwd}/data/', 'train_list': '${hydra:runtime.cwd}/data/train.txt', 'eval_lists_paths': ['${hydra:runtime.cwd}/data/train.txt'], 'eval_names': ['train'], 'test_list': '${hydra:runtime.cwd}/data/valid.txt', 'input_size': [128, 128], 'augmentation': {'module': 'landslide4sense.data.augmentations', 'name': 'transforms'}}, 'model': {'restore_from': None, 'module': 'segmentation_models_pytorch', 'name': 'UnetPlusPlus', 'args': {'in_channels': 14, 'classes': 2}}, 'optimizer': {'restore_from': None, 'module': 'torch.optim', 'name': 'Adam', 'args': {'lr': 0.001, 'weight_decay': 0.0005}}, 'loss': {'module': 'segmentation_models_pytorch.losses', 'name': 'DiceLoss', 'args': {'ignore_index': 255, 'mode': 'multiclass'}}, 'train': {'start_epoch': 0, 'steps_per_epoch': 125, 'batch_size': 128, 'num_workers': 2, 'num_steps': 10000, 'num_steps_stop': 10000, 'gpu_id': 0, 'snapshot_dir': '/content/drive/MyDrive/data_science_projects/landslide4sense-2022/models/', 'seed': 42

In [8]:
model_state_dict = torch.load(
    "../models/split_data_aug/epoch_7_val_f1=73.21207127753885.pth",  
    map_location=torch.device('cpu')
)
model = Unet(n_classes=2)
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [9]:
train_dataset = LandslideDataSet(
    "../data/",
    "../data/train_split_train.txt",
    max_iters=cfg.train.num_steps_stop * cfg.train.batch_size,
    set="labeled",
)

In [10]:
train_loader: DataLoader[LandslideDataSet] = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    num_workers=cfg.train.num_workers,
    pin_memory=True,
)